# Obtain df from images to view in Genuage

In [1]:
import os
import glob
from pathlib import Path 
import sys
import fnmatch
from itertools import cycle
from functools import partial
from math import log10, sqrt, ceil, floor, log, exp, pi, isnan
import numpy as np
import numpy.ma as ma
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.patches as mpatches
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
fontprops = fm.FontProperties(size=30)
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cmapy
import tifffile
import pandas as pd
import cv2
from scipy import fftpack
from scipy import ndimage
from scipy import stats
from scipy.spatial import distance
from scipy.cluster import hierarchy
from scipy.signal import convolve2d
import seaborn as sns
from tqdm.notebook import tqdm, trange
from skimage import color
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.manifold import TSNE
from skimage.transform import rescale
from skimage.util import view_as_windows
from skimage.feature import canny
from skimage.metrics import mean_squared_error as mse
from skimage.metrics import structural_similarity as ssim
from statannot import add_stat_annotation
import scipy
import pickle
import time

import multiprocessing as mp
from multiprocessing import Process, Queue, Value, Array, Pool

import matplotlib.font_manager as fm
%matplotlib inline
plt.rcParams["axes.grid"] = False
%gui qt5

In [2]:
# Import path
module_path = str(Path.cwd().parents[0])
if module_path not in sys.path:
    sys.path.append(module_path)

module_path = str(Path.cwd().parents[0] / "src")
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from my_io import display_list_image, read_tiff_files
from config import *

In [4]:
pickle_name_temp = f"{data_path_masked_BM}/image_list_BM.pickle"
zoomed_image_list_BM = pickle.load(open(pickle_name_temp, "rb"))

pickle_name_temp = f"{data_path_masked_UC}/image_list_UC.pickle"
zoomed_image_list_UC = pickle.load(open(pickle_name_temp, "rb"))

In [5]:
pickle_name_temp = f"{data_path_masked_BM}/mask_list_BM.pickle"
zoomed_mask_list_BM = pickle.load(open(pickle_name_temp, "rb"))

pickle_name_temp = f"{data_path_masked_UC}/mask_list_UC.pickle"
zoomed_mask_list_UC = pickle.load(open(pickle_name_temp, "rb"))

In [6]:
save_path = f"{data_dir}/vr_visualization/"
if not os.path.exists(save_path):
    os.makedirs(save_path)

### Loading the dataset

In [7]:
BM_indices = [2, 3, 8, 14, 18, 29, 30]
UC_indices = [0, 3, 9, 11, 19, 23, 30]

In [8]:
df_images_all_BM = pd.DataFrame()

q = 0.9

for x in trange(len(BM_indices)):
    m = BM_indices[x]
    pd_images = pd.DataFrame()
    images_temp = zoomed_image_list_BM[m].copy()
    masks_temp = zoomed_mask_list_BM[m].copy()/255
    
    array_i_index = np.zeros(images_temp[0].shape)
    array_j_index = np.zeros(images_temp[0].shape)
    for i in range(array_i_index.shape[0]):
        for j in range(array_i_index.shape[1]):
            array_i_index[i, j] = i
            array_j_index[i, j] = j
            
    pd_images_temp = pd.DataFrame(images_temp.reshape(11, -1).T, columns=channel_names)  
    pd_images_temp['cell'] = x
    pd_images_temp['mask'] = masks_temp.reshape(-1).copy()
    pd_images_temp['x'] = array_i_index.reshape(-1)*0.1083
    pd_images_temp['y'] = array_j_index.reshape(-1)*0.1083
    
    pd_images_temp = pd_images_temp[pd_images_temp['mask'] == 1.0]
    
    pd_images_temp = pd_images_temp[channel_names + ['cell', 'x', 'y']]
        
    df_images_all_BM = pd.concat([df_images_all_BM, pd_images_temp], axis=0)

  0%|          | 0/7 [00:00<?, ?it/s]

In [9]:
df_images_all_UC = pd.DataFrame()
q = 0.9

for x in trange(len(UC_indices)):
    m = UC_indices[x]
    pd_images = pd.DataFrame()
    images_temp = zoomed_image_list_UC[m].copy()
    masks_temp = zoomed_mask_list_UC[m].copy()/255
    
    array_i_index = np.zeros(images_temp[0].shape)
    array_j_index = np.zeros(images_temp[0].shape)
    for i in range(array_i_index.shape[0]):
        for j in range(array_i_index.shape[1]):
            array_i_index[i, j] = i
            array_j_index[i, j] = j
            
    pd_images_temp = pd.DataFrame(images_temp.reshape(11, -1).T, columns=channel_names)
    pd_images_temp['cell'] = x
    pd_images_temp['mask'] = masks_temp.reshape(-1).copy()
    pd_images_temp['x'] = array_i_index.reshape(-1)*0.1083
    pd_images_temp['y'] = array_j_index.reshape(-1)*0.1083
    
    pd_images_temp = pd_images_temp[pd_images_temp['mask'] == 1.0]
    
    pd_images_temp = pd_images_temp[channel_names + ['cell', 'x', 'y']]
        
    df_images_all_UC = pd.concat([df_images_all_UC, pd_images_temp], axis=0)

  0%|          | 0/7 [00:00<?, ?it/s]

### Selecting a cell

In [10]:
df_BM_cell = df_images_all_BM[df_images_all_BM['cell']==2]
df_BM_cell= df_BM_cell.drop(['cell'], axis=1)
df_BM_cell = df_BM_cell.melt(id_vars=['x','y'], value_vars=None, var_name='marker', value_name='c', col_level=None, ignore_index=True)
df_BM_cell

,x,y,marker,c
0,0.0000,124.2201,ATF6,14.031500
1,0.1083,124.0035,ATF6,5.737515
2,0.1083,124.1118,ATF6,7.580623
3,0.1083,124.2201,ATF6,14.215811
4,0.2166,123.8952,ATF6,10.160974
...,...,...,...,...
3723825,94.2210,47.1105,WGA,0.337324
3723826,94.2210,47.2188,WGA,1.573425
3723827,94.2210,47.3271,WGA,2.129670
3723828,94.3293,47.1105,WGA,1.573425


### ATF6

In [11]:
df_BM_atf6 = df_BM_cell[df_BM_cell['marker']=='ATF6']
df_BM_atf6 = df_BM_atf6.drop(['marker'], axis=1)
df_BM_atf6['z'] = 1
df_BM_atf6

,x,y,c,z
0,0.0000,124.2201,14.031500,1
1,0.1083,124.0035,5.737515,1
2,0.1083,124.1118,7.580623,1
3,0.1083,124.2201,14.215811,1
4,0.2166,123.8952,10.160974,1
...,...,...,...,...
338525,94.2210,47.1105,26.011700,1
338526,94.2210,47.2188,8.870798,1
338527,94.2210,47.3271,5.000272,1
338528,94.3293,47.1105,15.321675,1


In [12]:
df_BM_atf6.to_csv(save_path+'ATF6_BM_cell3.txt', header=None, index=None, sep='\t', mode='a')

### Beta Tubulin

In [13]:
df_BM_bt = df_BM_cell[df_BM_cell['marker']=='BetaTubulin']
df_BM_bt = df_BM_bt.drop(['marker'], axis=1)
df_BM_bt['z'] = 1
df_BM_bt

,x,y,c,z
338530,0.0000,124.2201,7.892065,1
338531,0.1083,124.0035,3.002618,1
338532,0.1083,124.1118,4.567241,1
338533,0.1083,124.2201,7.109754,1
338534,0.2166,123.8952,4.762819,1
...,...,...,...,...
677055,94.2210,47.1105,2.807040,1
677056,94.2210,47.2188,4.958397,1
677057,94.2210,47.3271,3.198196,1
677058,94.3293,47.1105,4.958397,1


In [14]:
df_BM_bt.to_csv(save_path+'BetaTubulin_BM_cell3.txt', header=None, index=None, sep='\t', mode='a')

### Concanavalin A

In [15]:
df_BM_cona = df_BM_cell[df_BM_cell['marker']=='ConcanavalinA']
df_BM_cona = df_BM_cona.drop(['marker'], axis=1)
df_BM_cona['z'] = 1
df_BM_cona

,x,y,c,z
677060,0.0000,124.2201,21.214579,1
677061,0.1083,124.0035,19.905448,1
677062,0.1083,124.1118,13.687077,1
677063,0.1083,124.2201,11.723381,1
677064,0.2166,123.8952,22.850992,1
...,...,...,...,...
1015585,94.2210,47.1105,19.578165,1
1015586,94.2210,47.2188,12.050663,1
1015587,94.2210,47.3271,14.668925,1
1015588,94.3293,47.1105,20.887296,1


In [16]:
df_BM_cona.to_csv(save_path+'ConcanavalinA_BM_cell3.txt', header=None, index=None, sep='\t', mode='a')

### DAPI

In [17]:
df_BM_dapi = df_BM_cell[df_BM_cell['marker']=='DAPI']
df_BM_dapi = df_BM_dapi.drop(['marker'], axis=1)
df_BM_dapi['z'] = 1
df_BM_dapi

,x,y,c,z
1015590,0.0000,124.2201,4.859372,1
1015591,0.1083,124.0035,13.218473,1
1015592,0.1083,124.1118,11.626263,1
1015593,0.1083,124.2201,11.228210,1
1015594,0.2166,123.8952,8.839896,1
...,...,...,...,...
1354115,94.2210,47.1105,16.800944,1
1354116,94.2210,47.2188,15.208735,1
1354117,94.2210,47.3271,14.412630,1
1354118,94.3293,47.1105,21.577573,1


In [18]:
df_BM_dapi.to_csv(save_path+'DAPI_BM_cell3.txt', header=None, index=None, sep='\t', mode='a')

### GOLPH4

In [19]:
df_BM_golph4 = df_BM_cell[df_BM_cell['marker']=='GOLPH4']
df_BM_golph4 = df_BM_golph4.drop(['marker'], axis=1)
df_BM_golph4['z'] = 1
df_BM_golph4

,x,y,c,z
1354120,0.0000,124.2201,0.794129,1
1354121,0.1083,124.0035,4.311607,1
1354122,0.1083,124.1118,2.603118,1
1354123,0.1083,124.2201,1.799123,1
1354124,0.2166,123.8952,1.698623,1
...,...,...,...,...
1692645,94.2210,47.1105,1.899622,1
1692646,94.2210,47.2188,3.005115,1
1692647,94.2210,47.3271,5.618099,1
1692648,94.3293,47.1105,1.698623,1


In [20]:
df_BM_golph4.to_csv(save_path+'GOLPH4_BM_cell3.txt', header=None, index=None, sep='\t', mode='a')

### HSP60

In [21]:
df_BM_hsp60 = df_BM_cell[df_BM_cell['marker']=='HSP60']
df_BM_hsp60 = df_BM_hsp60.drop(['marker'], axis=1)
df_BM_hsp60['z'] = 1
df_BM_hsp60

,x,y,c,z
1692650,0.0000,124.2201,3.219857,1
1692651,0.1083,124.0035,3.159346,1
1692652,0.1083,124.1118,4.611611,1
1692653,0.1083,124.2201,3.461901,1
1692654,0.2166,123.8952,3.643434,1
...,...,...,...,...
2031175,94.2210,47.1105,0.678392,1
2031176,94.2210,47.2188,0.617881,1
2031177,94.2210,47.3271,0.375837,1
2031178,94.3293,47.1105,0.678392,1


In [22]:
df_BM_hsp60.to_csv(save_path+'HSP60_BM_cell3.txt', header=None, index=None, sep='\t', mode='a')

### Nucleolin

In [23]:
df_BM_nuc = df_BM_cell[df_BM_cell['marker']=='Nucleolin']
df_BM_nuc = df_BM_nuc.drop(['marker'], axis=1)
df_BM_nuc['z'] = 1
df_BM_nuc

,x,y,c,z
2031180,0.0000,124.2201,0.273112,1
2031181,0.1083,124.0035,0.554961,1
2031182,0.1083,124.1118,0.000000,1
2031183,0.1083,124.2201,0.132187,1
2031184,0.2166,123.8952,0.038237,1
...,...,...,...,...
2369705,94.2210,47.1105,0.038237,1
2369706,94.2210,47.2188,0.000000,1
2369707,94.2210,47.3271,0.000000,1
2369708,94.3293,47.1105,0.000000,1


In [24]:
df_BM_nuc.to_csv(save_path+'Nucleolin_BM_cell3.txt', header=None, index=None, sep='\t', mode='a')

### Phalloidin

In [25]:
df_BM_ph = df_BM_cell[df_BM_cell['marker']=='Phalloidin']
df_BM_ph = df_BM_ph.drop(['marker'], axis=1)
df_BM_ph['z'] = 1
df_BM_ph

,x,y,c,z
2369710,0.0000,124.2201,12.602725,1
2369711,0.1083,124.0035,4.517958,1
2369712,0.1083,124.1118,9.511491,1
2369713,0.1083,124.2201,10.700427,1
2369714,0.2166,123.8952,9.035916,1
...,...,...,...,...
2708235,94.2210,47.1105,0.000000,1
2708236,94.2210,47.2188,0.951149,1
2708237,94.2210,47.3271,4.993533,1
2708238,94.3293,47.1105,1.902298,1


In [26]:
df_BM_ph.to_csv(save_path+'Phalloidin_BM_cell3.txt', header=None, index=None, sep='\t', mode='a')

### Sortilin

In [27]:
df_BM_sor = df_BM_cell[df_BM_cell['marker']=='Sortilin']
df_BM_sor = df_BM_sor.drop(['marker'], axis=1)
df_BM_sor['z'] = 1
df_BM_sor

,x,y,c,z
2708240,0.0000,124.2201,2.732910,1
2708241,0.1083,124.0035,0.993363,1
2708242,0.1083,124.1118,2.437515,1
2708243,0.1083,124.2201,2.503158,1
2708244,0.2166,123.8952,2.207764,1
...,...,...,...,...
3046765,94.2210,47.1105,1.781082,1
3046766,94.2210,47.2188,3.061126,1
3046767,94.2210,47.3271,2.404694,1
3046768,94.3293,47.1105,1.584153,1


In [28]:
df_BM_sor.to_csv(save_path+'Sortilin_BM_cell3.txt', header=None, index=None, sep='\t', mode='a')

### TOM20

In [29]:
df_BM_t20 = df_BM_cell[df_BM_cell['marker']=='TOM20']
df_BM_t20 = df_BM_t20.drop(['marker'], axis=1)
df_BM_t20['z'] = 1
df_BM_t20

,x,y,c,z
3046770,0.0000,124.2201,5.215022,1
3046771,0.1083,124.0035,5.462038,1
3046772,0.1083,124.1118,3.485910,1
3046773,0.1083,124.2201,2.744862,1
3046774,0.2166,123.8952,4.473974,1
...,...,...,...,...
3385295,94.2210,47.1105,0.274702,1
3385296,94.2210,47.2188,1.015750,1
3385297,94.2210,47.3271,3.238894,1
3385298,94.3293,47.1105,8.920262,1


In [30]:
df_BM_t20.to_csv(save_path+'TOM20_BM_cell3.txt', header=None, index=None, sep='\t', mode='a')

### WGA

In [31]:
df_BM_wga = df_BM_cell[df_BM_cell['marker']=='WGA']
df_BM_wga = df_BM_wga.drop(['marker'], axis=1)
df_BM_wga['z'] = 1
df_BM_wga

,x,y,c,z
3385300,0.0000,124.2201,6.826853,1
3385301,0.1083,124.0035,5.590752,1
3385302,0.1083,124.1118,5.158117,1
3385303,0.1083,124.2201,3.922016,1
3385304,0.2166,123.8952,5.714362,1
...,...,...,...,...
3723825,94.2210,47.1105,0.337324,1
3723826,94.2210,47.2188,1.573425,1
3723827,94.2210,47.3271,2.129670,1
3723828,94.3293,47.1105,1.573425,1


In [32]:
df_BM_wga.to_csv(save_path+'WGA_BM_cell3.txt', header=None, index=None, sep='\t', mode='a')